# NordBord Hamstring Data

### This code will output a report of potential hamstring risk for each player after NordBord testing.

In [25]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime
import openpyxl
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter

In [26]:
# loading in data file and extract name, date, average L/R force, average imbalance columns
df = pd.read_excel("C:\\Users\\Katrina Peralta\\VT Football Sports Science 2024-2025\\Nordbord\\Nordbord Data.xlsx", usecols = 'A, C, I, J, K')   # extracting specific columns

# separating athletes
player_df = df.groupby(['Name']) # group data by athlete
players_list =  sorted(player_df.groups.keys()) # list of player names

# Making date column in datetime format
df['Date UTC'] = pd.to_datetime(df['Date UTC'], errors='coerce')


### Metrics Over Last 4 Tests

In [34]:
results = []

# redact player names
player_num = {player: f"{i:03}" for i, player in enumerate(players_list, start=1)}

# go through each player's data
for player in players_list:
    player_data = player_df.get_group((player,)).sort_values('Date UTC', ascending=True) # get data for the player

    # Checking for enough data points
    if len(player_data) < 4:
        #print(f"Not enough data points for {player}.")
        continue

    # Check for NaN values
    if player_data[['L Max Force (N)', 'R Max Force (N)', 'Max Imbalance (%)']].isnull().any().any():
        print(f"NaN values detected in {player}'s data.")
        continue

    # computing metrics
    for column in ['L Max Force (N)', 'R Max Force (N)', 'Max Imbalance (%)']:
        # rolling avg
        player_data[f'{column} Rolling Avg'] = (
            player_data[column]
            .rolling(window=4, min_periods=1)
            .mean()
            .round(2)
        )
        
        # rolling std calculated from the original column, not from the rolling average
        player_data[f'{column} Rolling Std'] = (
            player_data[column]
            .rolling(window=4, min_periods=1)
            .std()
            .round(2)
        )

         # fixing misalignment by shifting rolling std forward if needed
        player_data[f'{column} Rolling Std'] = player_data[f'{column} Rolling Std'].bfill()


        # upper bound
        player_data[f'{column} Upper Bound'] = (
            player_data[f'{column} Rolling Avg'] + player_data[f'{column} Rolling Std']
            ).round(2)
        
        # lower bound
        player_data[f'{column} Lower Bound'] = (
            player_data[f'{column} Rolling Avg'] - player_data[f'{column} Rolling Std']
            ).round(2)

        # std drop
        player_data[f'{column} STD Drop'] = player_data[f'{column} Rolling Std'].diff().round(2)

        # checking if in upper/lower bounds
        player_data[f'{column} Within Bounds'] = (
            (player_data[column] >= player_data[f'{column} Lower Bound']) &
            (player_data[column] <= player_data[f'{column} Upper Bound'])
        )

    # only select most recent 4 tests
    last_four = player_data.tail(4)

    # Debugging: Check if rolling calculations are working
    #print(last_four[['Date UTC', 'L Max Force (N)', 'L Max Force (N) Rolling Avg', 'L Max Force (N) Rolling Std']])


    # checking for 10-15% imbalance in any of last 4 tests and outputting weaker side
    for _, row in last_four.iterrows():
        imbalance = row['Max Imbalance (%)']
        imbalance_side = 'Left Stronger' if imbalance < 0 else 'Right Stronger'

        player_data['L Max Force (N) STD Drop'] = player_data['L Max Force (N) Rolling Std'].diff().round(2)
        player_data['R Max Force (N) STD Drop'] = player_data['R Max Force (N) Rolling Std'].diff().round(2) 
        
        sig_decrease = (
            row['L Max Force (N) STD Drop'] < -10 or 
            row['R Max Force (N) STD Drop'] < -10
        )
        results.append({
            'Player': player_num[player],
            'Date': row['Date UTC'].date(),
            'L Max Force': row['L Max Force (N)'],
            'R Max Force': row['R Max Force (N)'],
            'L Max Force Rolling Avg': row['L Max Force (N) Rolling Avg'],
            'R Max Force Rolling Avg': row['R Max Force (N) Rolling Avg'],
            'L Max Force Rolling Std': row['L Max Force (N) Rolling Std'],
            'R Max Force Rolling Std': row['R Max Force (N) Rolling Std'],
            'L Max Force Upper Bound': row['L Max Force (N) Upper Bound'],
            'L Max Force Lower Bound': row['L Max Force (N) Lower Bound'],
            'L Force Within Bounds': row['L Max Force (N) Within Bounds'],
            'R Max Force Upper Bound': row['R Max Force (N) Upper Bound'],
            'R Max Force Lower Bound': row['R Max Force (N) Lower Bound'],
            'R Force Within Bounds': row['R Max Force (N) Within Bounds'],
            'Max Imbalance (%)': round(row['Max Imbalance (%)'], 2),
            'Imbalance Side': imbalance_side,
            'Significant Force Decrease': sig_decrease
        })

# Putting data into a table
results_df = pd.DataFrame(results)

# Exporting to excel file
results_df.to_excel("C:\\Users\\Katrina Peralta\\VT Football Sports Science 2024-2025\\NordBordNewList.xlsx", index=False)

### Applying Risk Criteria

In [35]:
# determining risk criteria
def determine_risk(row):
    # checking if below lower bound
    below_lower_bound = (
        row['L Max Force'] < row['L Max Force Lower Bound'] or 
        row['R Max Force'] < row['R Max Force Lower Bound'] 
    )

    # calculating risk factors
    max_imbalance = abs(row['Max Imbalance (%)'])
    sig_decrease = row['Significant Force Decrease']

    # criteria logic
    high_risk = sum([
        max_imbalance >= 15,
        sig_decrease 
    ])

    if high_risk >= 2:
        return 'High Risk'
    elif sig_decrease or max_imbalance >= 15:
        return 'Medium Risk'
    elif below_lower_bound or (10 <= max_imbalance < 15):
        return 'Low Risk'
    else:
        return 'No Risk'

# apply risk categories to dataframe
results_df['Risk Level'] = results_df.apply(determine_risk, axis=1)

from openpyxl.styles import PatternFill

# apply conditional formatting for risk levels
color_mapping = {
    "High Risk": "FF0000",  # Red
    "Medium Risk": "FFA500",  # Orange
    "Low Risk": "FFFF00",  # Yellow
    "No Risk": "FFFFFF"  # White
}

# define file path for the output
output_file = "C:\\Users\\Katrina Peralta\\VT Football Sports Science 2024-2025\\Nordbord\\NordBord_Risk_Report.xlsx"

# save and apply formatting for color coding
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # write the DataFrame
    results_df.to_excel(writer, index=False, sheet_name="Summary")

    # access workbook and worksheet
    workbook = writer.book
    worksheet = writer.sheets["Summary"]

    # get the column letter for "Risk Level"
    risk_col_idx = results_df.columns.get_loc("Risk Level") + 1
    risk_col_letter = get_column_letter(risk_col_idx)

    # apply color coding to each row
    for idx, risk_level in enumerate(results_df['Risk Level'], start=2):
        color = color_mapping.get(risk_level, "FFFFFF")
        cell_address = f"{risk_col_letter}{idx}"
        cell = worksheet[cell_address]
        cell.fill = PatternFill(start_color=color, end_color=color, fill_type="solid")

      

### Dashboard

In [36]:
# importing packages
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_table
import plotly.graph_objects as go
import webbrowser

# load processed Excel data
df = pd.read_excel(output_file, sheet_name='Summary')

# initialize Dash app
app = dash.Dash(__name__)

server = app.server

# define layout of dashboard
app.layout = html.Div([
    html.H1("NordBord Analysis Dashboard"),
    
    # dropdown menu for PlayerID
    html.Label('Select a Player:'),
    dcc.Dropdown(
        id='player-dropdown',
        options=[{'label': player, 'value': player} for player in sorted(df['Player'].unique())],
        value=sorted(df['Player'].unique())[0],
        clearable=False
    ),

    # risk table of each risk level for each test date
    html.H2('Risk Levels Per Test'),
    dash_table.DataTable(
        id='risk-level-table',
        columns=[
            {'name': 'Date', 'id': 'Date'},
            {'name': 'Risk Level', 'id': 'Risk Level'}
        ],
        style_table={'height': '350px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'center', 'padding': '10px'},
        style_data_conditional=[
        {
            'if': {
                'filter_query': '{Risk Level} = "High Risk"',
                'column_id': 'Risk Level'
            },
            'backgroundColor': 'red',
            'color': 'white'
        },
        {
            'if': {
                'filter_query': '{Risk Level} = "Medium Risk"',
                'column_id': 'Risk Level'
            },
            'backgroundColor': 'orange',
            'color': 'white'
        },
        {
            'if': {
                'filter_query': '{Risk Level} = "Low Risk"',
                'column_id': 'Risk Level'
            },
            'backgroundColor': 'yellow',
            'color': 'black'
        },
        {
            'if': {
                'filter_query': '{Risk Level} = "No Risk"',
                'column_id': 'Risk Level'
            },
            'backgroundColor': 'white',
            'color': 'black'
        }
    ]

    ),

    # force metrics over time
    html.H2('Force Metrics Over Time'),
    dcc.Graph(id='force-metrics-graph'),
    
    # summary statistics table
    html.H2('Summary Statistics'),
    dcc.Graph(id='summary-stats-table'),
])

# define callback functions to update table and graphs based on player dropdown selection
@app.callback(
   [Output('force-metrics-graph', 'figure'),
    Output('risk-level-table', 'data'),
    Output('summary-stats-table', 'figure')],
    [Input('player-dropdown', 'value')]
) 


def update_graphs(selected_player):
    player_data = df[df['Player'] == selected_player]

    # create figure
    force_fig = go.Figure()

    # adding L Max Force line
    force_fig.add_trace(go.Scatter(
        x=player_data['Date'],
        y=player_data['L Max Force'],
        mode='lines',
        name='L Max Force',
        line=dict(color='blue')
    ))

    # Add L Max Force +/- std 
    force_fig.add_trace(go.Scatter(
        x=player_data['Date'].tolist() + player_data['Date'].tolist()[::-1],
        y=(player_data['L Max Force'] + 0.5*player_data['L Max Force Rolling Std']).tolist() + 
          (player_data['L Max Force'] - 0.5*player_data['L Max Force Rolling Std']).tolist()[::-1],
           fill='toself',
           fillcolor='rgba(0,0,255,0.2)',
           line=dict(color='rgba(255,255,255,0)'),
           showlegend=False   
    ))

    # adding R Max Force line
    force_fig.add_trace(go.Scatter(
        x=player_data['Date'],
        y=player_data['R Max Force'],
        mode='lines',
        name='R Max Force',
        line=dict(color='red')
    ))

    # add R Max Force line +/- std
    force_fig.add_trace(go.Scatter(
        x=player_data['Date'].tolist() + player_data['Date'].tolist()[::-1],
        y=(player_data['R Max Force'] + 0.5*player_data['R Max Force Rolling Std']).tolist() + 
          (player_data['R Max Force'] - 0.5*player_data['R Max Force Rolling Std']).tolist()[::-1],
           fill='toself',
           fillcolor='rgba(255,0,0,0.2)',
           line=dict(color='rgba(255,255,255,0)'),
           showlegend=False   
    ))

    # formatting
    force_fig.update_layout(
        title='Left vs Right Max Force Over Time',
        xaxis_title='Date',
        yaxis_title='Force (N)',
        yaxis=dict(range=[0, player_data[['L Max Force', 'R Max Force']].max().max() * 1.1]),  # adjust y-axis range
        template='plotly_white',
        height=700,
        width=1200
    )

    # risk level table
    risk_level_table = [
        {
            'Date': row['Date'].strftime('%Y-%m-%d'),
            'Risk Level': row['Risk Level']
        }
        for _, row in player_data.iterrows()
    ]

    # summary stats table
    summary_stats = player_data[['L Max Force', 'R Max Force', 'Max Imbalance (%)']].describe()
    summary_fig = px.imshow(summary_stats, text_auto=True, title='Summary Statistics')

    return force_fig, risk_level_table, summary_fig

# Run Dash app
if __name__ == '__main__':
    webbrowser.open_new('http://127.0.0.1:8050/')   # opens Dash app in browser
    app.run_server(debug=True)